In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction'

In [3]:
import urllib.request as request

filename, headers = request.urlretrieve(
    url="https://docs.google.com/spreadsheets/d/1taus8Wx2s9xcavripC_TMuDNF2vVSVZ_/edit?usp=sharing&ouid=113748405694445668234&rtpof=true&sd=true",
    filename="sample_data__technical_assessment.xlsx"
)

In [3]:
import pandas as pd

dff = pd.read_excel(
    'artifacts/data_ingestion/sample_data__technical_assessment.xlsx',
    engine='openpyxl'
)

In [29]:
dff.head(10)

,activity_month,account_id,brand_id,reg_date,ftd_date,qp_date,ben_login_id,tracker_id,player_reg_product,total_deposit,total_handle,total_ngr
0,2023-11-01,185196607,PA,2023-11-21,2023-11-21,NaT,BCUSA22L,7102592,BETTING,10.0,210.00,13.18
1,2023-11-01,154497623,AZ,2022-10-28,2022-10-28,2022-10-28,actionnetwork,7039239,BETTING,0.0,5.00,4.00
2,2023-11-01,153249011,MI,2022-08-18,2023-12-21,NaT,starsportsmx,7049874,CASINO,0.0,0.00,0.00
3,2023-11-01,184718245,OH,2023-11-13,2023-11-13,2023-11-13,actionnetwork,7106987,BETTING,10.0,210.00,138.98
4,2023-11-01,183573211,OH,2023-11-06,2023-11-06,2023-11-06,BCUSA22NP,7106983,BETTING,10.0,210.00,-320.22
5,2023-11-01,153923970,PA,2022-09-12,2022-09-12,2022-09-12,BCUSA22N,7086406,CASINO,0.0,0.00,0.00
6,2023-11-01,174413334,TN,2023-10-05,2023-10-05,2023-11-21,BCUSA22N,7086400,BETTING,60.0,303.00,68.46
7,2023-11-01,153776081,AZ,2022-09-10,2022-09-10,2022-09-15,starsportsmx,7080479,BETTING,0.0,170.00,136.00
8,2023-11-01,160312091,OH,2022-12-03,2023-05-20,2023-01-08,actionnetwork,7083318,BETTING,0.0,1188.58,261.74
9,2023-11-01,184803642,MD,2023-11-14,NaT,NaT,actionnetwork,7106987,BETTING,0.0,0.00,0.00


In [ ]:
for id in dff['account_id'].unique():
    filtered_df = dff.query("account_id == @id")
    
    if len(filtered_df) > 1:
        print(filtered_df)
        print('________________________________________________________________________________________')

In [48]:
df = dff.copy()
date_columns = ['activity_month', 'reg_date', 'ftd_date', 'qp_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [51]:
df['months_active'] = ((df['activity_month'] - df['ftd_date']).dt.days / 30).fillna(0).astype(int)
df_agg = df.groupby(['account_id', 'activity_month']).agg({
        'months_active': 'max',
        'total_deposit': 'sum',
        'total_handle': 'sum',
        'total_ngr': 'sum',
        'brand_id': 'first',
        'ben_login_id': 'first',
        'player_reg_product': 'first',
        'reg_date': 'first'
    }).reset_index()
# df = df.drop(columns=['activity_month'])

In [24]:
n = dff.isnull().sum()

n = n[n > 0]

In [26]:
n.index.to_list()

['ftd_date', 'qp_date', 'total_handle']

In [20]:
print(list(dff.columns))

['activity_month', 'account_id', 'brand_id', 'reg_date', 'ftd_date', 'qp_date', 'ben_login_id', 'tracker_id', 'player_reg_product', 'total_deposit', 'total_handle', 'total_ngr']


In [32]:
from sklearn.preprocessing import OneHotEncoder

In [34]:
categorical_features = dff.select_dtypes(include='object').columns.to_list()

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = pd.DataFrame(
    encoder.fit_transform(dff[categorical_features]),
    columns=encoder.get_feature_names_out(categorical_features),
    index=dff.index
)

df = pd.concat([dff, encoded], axis=1)
df.drop(categorical_features, axis=1, inplace=True)

In [36]:
df.head()

,activity_month,account_id,reg_date,ftd_date,qp_date,tracker_id,total_deposit,total_handle,total_ngr,brand_id_AZ,...,ben_login_id_BCUSA22N,ben_login_id_BCUSA22NP,ben_login_id_actionnetwork,ben_login_id_starsportsmx,ben_login_id_wedgetraffic,player_reg_product_BETTING,player_reg_product_BINGO,player_reg_product_CASINO,player_reg_product_POKER,player_reg_product_RACEBOOK
0,2023-11-01,185196607,2023-11-21,2023-11-21,NaT,7102592,10.0,210.0,13.18,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2023-11-01,154497623,2022-10-28,2022-10-28,2022-10-28,7039239,0.0,5.0,4.00,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2023-11-01,153249011,2022-08-18,2023-12-21,NaT,7049874,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2023-11-01,184718245,2023-11-13,2023-11-13,2023-11-13,7106987,10.0,210.0,138.98,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2023-11-01,183573211,2023-11-06,2023-11-06,2023-11-06,7106983,10.0,210.0,-320.22,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [43]:
import numpy as np

numeric_features = df.select_dtypes(include=np.float64).columns.to_list()

In [46]:
dff.columns.

Index(['activity_month', 'account_id', 'brand_id', 'reg_date', 'ftd_date',
       'qp_date', 'ben_login_id', 'tracker_id', 'player_reg_product',
       'total_deposit', 'total_handle', 'total_ngr'],
      dtype='object')